# Setup

In [1]:
# %pip install google-adk -q
# %pip install litellm -q

print("Installation complete.")


Installation complete.


### Importing Dependencies

In [2]:
# @title Import necessary libraries
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

Libraries imported.


### Configuring API Keys

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


# --- Verify Keys ---
print("API Keys Set:")
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

# Configure ADK to use API keys directly (not Vertex AI for this multi-model setup)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"


API Keys Set:
Google API Key set: Yes


### Defining Model Constants

In [4]:
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

# This seemed to work better
MODEL_GEMINI_2_0_FLASH_EXP = "gemini-2.0-flash-exp"


print("\nEnvironment configured.")


Environment configured.


# Building First Agent

**Core Principles**

- Agent: The underlying "brain" that communicates with the user and determines what to do depending on user requests
- Tool: Python functions that gives the agent to perform specific tasks. We can have functions to check time, look up weather, send emails, etc. Tools should be detailed and very systematic


### Defining Tools

**Key Concepts**: Docstrings are crucial, as good ones allow agents to better under **how** to use them and to understand:
- what the tool does
- when to use it
- what arguments it requires
- what information it returns

**Best Practice**: Write clear, descriptive, and accurate docstring for tools. (There's a potential for ChatGPT to help write the docstrings, since they're capable of being descriptive)

In [5]:
# @title Define the get_weather Tool
def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "New York", "London", "Tokyo").

    Returns:
        dict: A dictionary containing the weather information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with weather details.
              If 'error', includes an 'error_message' key.
    """
    print(f"--- Tool: get_weather called for city: {city} ---") # Log tool execution
    city_normalized = city.lower().replace(" ", "") # Basic normalization

    # Mock weather data
    mock_weather_db = {
        "newyork": {"status": "success", "report": "The weather in New York is sunny with a temperature of 25°C."},
        "london": {"status": "success", "report": "It's cloudy in London with a temperature of 15°C."},
        "tokyo": {"status": "success", "report": "Tokyo is experiencing light rain and a temperature of 18°C."},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have weather information for '{city}'."}

# Example tool usage (optional test)
print(get_weather("New York"))
print(get_weather("Paris"))

--- Tool: get_weather called for city: New York ---
{'status': 'success', 'report': 'The weather in New York is sunny with a temperature of 25°C.'}
--- Tool: get_weather called for city: Paris ---
{'status': 'error', 'error_message': "Sorry, I don't have weather information for 'Paris'."}


# Defining Agents

An orchestrator that facilitates interaction between the user and LLM and available tools

For ADK, there's several key parameters for an agent:
- `name`: A unique identifier for this agent
- `model`: Specifies what model this agent should use. Different models may be more capable than others had doing certain tasks
- `description`: A concise sumnmary of the agent's overall purpose, which is crucial later when other agents need to decide whether to delegate tasks to this agent.
- `instruction`: Detailed guidance for the LLM on how to behave, its persona, its goals, and specifically how and when to utilize its assigned `tools`
- `tools`: A list containing the actual Python tool functions that the agent is allowed to use

**Best Practice**: Choose descriptive `name` and `description` values, since those are used internally by ADK and are vital for features like automic delegation

**Note**: In a way, the agents are structured like a tree hierachy, where task are first passed to the root agent, then it gets sent to respective agents that are most suited for the task. An analogy can be like a company and how tasks are dealt with. The CEO sees a tasks, then it decides to pass it to a particular department, then the department leader pass to a subgroup to handle

In [6]:
# @title Define the Weather Agent
# Use one of the model constants defined earlier
AGENT_MODEL = MODEL_GEMINI_2_0_FLASH

weather_agent = Agent(
    name="weather_agent_v1",
    model=AGENT_MODEL, # Can be a string for Gemini or a LiteLlm object
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather], # Pass the function directly
)

print(f"Agent '{weather_agent.name}' created using model '{AGENT_MODEL}'.")

Agent 'weather_agent_v1' created using model 'gemini-2.0-flash'.


### Setup Runner and Session Servicess

This is to manage conversation and execute the agent, and has 2 components:

- `SessionService`: Responsible for managing conversation history and state for different user and sessions. In the following, we'll use `InMemorySessionService`, a simple implementation that stores everything in memory, suitable for testing and simple applications. It also keeps track of messages exchanged
- `Runner`: The engine that orchestrates the interaction flow. It takes user input, route it to the appropriate agent, manages calls to the LLM and tools based on the agent's logic, handles session updates via the `SessionService`, and yields events representing the process of the interaction

In short, the `SessionService` manages conversation history for different users, and the `Runner` is the facilitator that direct the flow of user inputs, determining which sub-agents are the most appropriate for a particular task



In [7]:

session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=weather_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='weather_tutorial_app', User='user_1', Session='session_001'
Runner created for agent 'weather_agent_v1'.


Note: When looking at the above code, it seems like Runner is given an agent and a session. Thus the runner is specific to that particular session, as well as having access to that specific agent

### Interacting with the Agent



We need a way to send messages to our agents and recieve its responses. Since LLM calls and tool execution can take time, ADK's `Runner` operates asynchronously. 

A helper function `call_agent_async` will be used to:
- Takes a a user query string
- Packages it into ADK `Content` format (Its expected input)
- Calls `runner.run.async`, providing the user/session context and the new message
- Iterates through the Event yielded by the runner. Events represent steps in the agent's execution (e.g. tool call requested, tool result recieved, intermediate LLM thought, final response)
- Identifies and prints the final response event using `event.is_final_response()`

**Purpose of `async`**: Interactions with LLMs and some tools are I/O-bound operations. Using `asyncio` allows the program to handle these operations efficiently without blocking execution

In [8]:
# @title Define Agent Interaction Function

from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

### Running the Conversation

Since `call_agent_async` is asynchronous, we don't want the output to mix the text between multiple agent calls. That's why we have the `await` keyword to tell python to wait until the response is fully done to move up to the next call

In [9]:
# @title Run the Initial Conversation

async def run_conversation():
    await call_agent_async("What is the weather like in London?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    await call_agent_async("How about Paris?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID) # Expecting the tool's error message

    await call_agent_async("Tell me the weather in New York",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()



>>> User Query: What is the weather like in London?
--- Tool: get_weather called for city: London ---
<<< Agent Response: The weather in London is cloudy with a temperature of 15°C.


>>> User Query: How about Paris?
--- Tool: get_weather called for city: Paris ---
<<< Agent Response: I am sorry, I don't have weather information for Paris.


>>> User Query: Tell me the weather in New York
--- Tool: get_weather called for city: New York ---
<<< Agent Response: The weather in New York is sunny with a temperature of 25°C.



# Building an Agent Team

In [23]:
# These will be the tools that the agents can use

def say_hello(name: str = "World") -> str:
    """Provides a simple greeting for a given name

    Args:
        name (str, optional): The name of the person to greet.

    Returns:
        str: A friendly greeting message
    """
    print(f"--- Tool: [say_hello] called with name: {name} ---")
    
    return f"Hello, {name}!"

def say_goodbye(name: str = "World") -> str:
    """Provides a goodbye message for a given name

    Args: 
        name (str, optional): The name of the person to bid farewell

    Returns:
        str: A farewell message to conclude the conversation
    """

    print(f"--- Tool: [say_goodbye] called with name: {name} ---")
    
    return f"Goodbye, {name}! Have a splendid day!"

print("Greeting and Farewell tools defined")


Greeting and Farewell tools defined


In [11]:
# --- Greeting Agent ---
greeting_agent = None
try:
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent",
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting to the user. "
                    "Use the 'say_hello' tool to generate the greeting. "
                    "Do NOT engage in any other conversations or tasks. ",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Agent '{greeting_agent.name}' created using model '{greeting_agent.model}'.")
except Exception as e:
    print(f"❌ Could not create Greeting agent. Check API Key ({greeting_agent.model}). Error: {e}")

# --- Farewell Agent ---
farewell_agent = None
try:
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent",
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message to the user. "
                    "Use the 'say_goodbye' tool when the user indicates they are leaving or ending the conversation. "
                    "Do NOT engage in any other conversations or tasks. ",
        description="Handles farewell message by using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Agent '{farewell_agent.name}' created using model '{farewell_agent.model}'.")
except Exception as e:
    print(f"❌ Could not create Farewell agent. Check API Key ({farewell_agent.model}). Error: {e}")

✅ Agent 'greeting_agent' created using model 'gemini-2.0-flash'.
✅ Agent 'farewell_agent' created using model 'gemini-2.0-flash'.


Notes: Clear and concise descriptions are cruical for task delegation. When an agent is chosen, then the instructions are taken into consideration. 

In [18]:
root_agent = None
runner_root = None

if greeting_agent and farewell_agent and 'get_weather' in globals():
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    weather_agent_team = Agent(
        name="weather_agent_v2",
        model=root_agent_model,
        description="The main coordinator agent. Handles weather requests and delegates greetings/farewells to specialists. ",
        instruction="You are the main Weather Agent coordinating a team. Your primary responsibility is to provide weather information. "
                    "Use the 'get_weather' tool ONLY for specific weather requests (e.g., 'weather in London'). "
                    "You have 2 specialized sub-agents: "
                    "1. 'greeting_agent': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. "
                    "2. 'farewell_agent': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. "
                    "Analyze the user's query. If it's a greeting, delegate to 'greeting_agent'. If it's a farewell, delegate to 'farewell_agent'. "
                    "If it's a weather request, handle it yourself using 'get_weather'. "
                    "For anything else, respond appropriately or state you cannot handle it.",
        tools= [get_weather],
        sub_agents=[greeting_agent, farewell_agent]
    )
    print(f"✅ Root Agent '{weather_agent_team.name}' created using model '{root_agent_model}' with sub-agents: {[sa.name for sa in weather_agent_team.sub_agents]}")

else:
    print("❌ Cannot create root agent because one or more sub-agents failed to initialize or 'get_weather' tool is missing.")
    if not greeting_agent: print(" - Greeting Agent is missing.")
    if not farewell_agent: print(" - Farewell Agent is missing.")
    if 'get_weather' not in globals(): print(" - get_weather function is missing.")




✅ Root Agent 'weather_agent_v2' created using model 'gemini-2.0-flash' with sub-agents: ['greeting_agent', 'farewell_agent']


In [29]:
import asyncio

root_agent_var_name = 'root_agent'

if 'weather_agent_team' in globals():
    root_agent_var_name = 'weather_agent_team'
elif 'root_agent' not in globals():
    print("⚠️ Root agent ('root_agent' or 'weather_agent_team') not found. Cannot define run_team_conversation.")
    root_agent = None

# First condition checks if the var_name exist, and second checks if that value isn't none
if root_agent_var_name in globals() and globals()[root_agent_var_name]:

    async def run_team_conversation():
        print("\n--- Testing Agent Team Delegation ---")
        
        APP_NAME = "weather_tutorial_agent_team"
        USER_ID = "user_1_agent_team"
        SESSION_ID = "session_001_agent_team"
        session_service = InMemorySessionService()
        session = session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
        )

        print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

        actual_root_agent = globals()[root_agent_var_name]
        runner_agent_team = Runner(
            agent=actual_root_agent,
            app_name=APP_NAME,
            session_service=session_service
        )
        print(f"Runner created for agent '{actual_root_agent.name}'.")

         # --- Interactions using await (correct within async def) ---
        await call_agent_async(query = "Hello there!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "What is the weather in New York?",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "Thanks, bye!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        
    print("Attempting executing using 'await'...")
    await run_team_conversation()



Attempting executing using 'await'...

--- Testing Agent Team Delegation ---
Session created: App='weather_tutorial_agent_team', User='user_1_agent_team', Session='session_001_agent_team'
Runner created for agent 'weather_agent_v2'.

>>> User Query: Hello there!
--- Tool: [say_hello] called with name: World ---
<<< Agent Response: Hello, World!


>>> User Query: What is the weather in New York?
--- Tool: get_weather called for city: New York ---
<<< Agent Response: The weather in New York is sunny with a temperature of 25°C.


>>> User Query: Thanks, bye!
--- Tool: [say_goodbye] called with name: World ---
<<< Agent Response: Goodbye, World! Have a splendid day!

